In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
import copy
from pandas.plotting import register_matplotlib_converters

import copy
import os

register_matplotlib_converters()
os.getcwd()

'/Volumes/GoogleDrive/My Drive/DataReleasePaper/ReleaseData/2-DryCreek'

In [2]:
import netCDF4 as nc4            # to work with netCDFs
import numpy   as np             # to perform numerics
import time

### Change the name of the watershed

In [3]:
watershed= 'DryCreek'

In [4]:
# Read the table
table = pd.read_csv('5_'+watershed+'_final_cleaned.csv',
                    header = 0,index_col = 'DateTime',parse_dates = True, 
                    infer_datetime_format = True,low_memory=False)


In [5]:
table.head(2)

,TL_Discharge,BSG_Discharge,LG_Discharge,C1E_Discharge,C1W_Discharge,C2E_Discharge,C2M_Discharge,BRW_Precipitation,LDP_Precipitation,SCR_Precipitation,...,LS_SoilTemperature_Pit1_30cm,LS_SoilTemperature_Pit2_2cm,LS_SoilTemperature_Pit2_15cm,LS_SoilTemperature_Pit2_30cm,LS_SoilTemperature_Pit3_2cm,LS_SoilTemperature_Pit3_15cm,LS_SoilTemperature_Pit3_30cm,LS_SoilTemperature_Pit4_2cm,LS_SoilTemperature_Pit4_15cm,LS_SoilTemperature_Pit4_30cm
DateTime,,,,,,,,,,,,,,,,,,,,,
1999-01-01,NaN,NaN,7.636792,29.606656,NaN,2.169073,217.072885,NaN,NaN,NaN,...,7.416064,4.911765,6.497507,7.179774,4.824208,7.195240,7.837034,5.403277,6.128272,6.495671
1999-01-02,NaN,NaN,44.185813,8.396678,NaN,2.251513,9.084700,NaN,NaN,NaN,...,7.286968,4.758782,6.352038,7.046998,4.669668,7.051027,7.709140,5.255897,5.992621,6.367365


In [6]:
table_flag =  pd.read_csv('5_'+watershed+'_final_cleaned_flag.csv',
                    header = 0,index_col = 'DateTime',parse_dates = True, 
                    infer_datetime_format = True,low_memory=False)


In [7]:
table_flag.head(2)

,TL_Discharge,BSG_Discharge,LG_Discharge,C1E_Discharge,C1W_Discharge,C2E_Discharge,C2M_Discharge,BRW_Precipitation,LDP_Precipitation,SCR_Precipitation,...,LS_SoilTemperature_Pit1_30cm,LS_SoilTemperature_Pit2_2cm,LS_SoilTemperature_Pit2_15cm,LS_SoilTemperature_Pit2_30cm,LS_SoilTemperature_Pit3_2cm,LS_SoilTemperature_Pit3_15cm,LS_SoilTemperature_Pit3_30cm,LS_SoilTemperature_Pit4_2cm,LS_SoilTemperature_Pit4_15cm,LS_SoilTemperature_Pit4_30cm
DateTime,,,,,,,,,,,,,,,,,,,,,
1999-01-01,NaN,NaN,4.0,4.0,NaN,4.0,4.0,NaN,NaN,NaN,...,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0
1999-01-02,NaN,NaN,4.0,4.0,NaN,4.0,4.0,NaN,NaN,NaN,...,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0


### Create NetCDF file

In [8]:
ncid = nc4.Dataset(watershed + "_NetCDF.nc", "w", format="NETCDF4")

### Extract the data from csv file

In [9]:
#### 1. Extract grid points for each met variable

In [10]:
all_stations=table.columns.str.extract(r'([^_]+)')[0]
print(all_stations.unique())
print ('  ')

['TL' 'BSG' 'LG' 'C1E' 'C1W' 'C2E' 'C2M' 'BRW' 'LDP' 'SCR' 'LW' 'HN' 'HS'
 'MHN' 'MHS' 'MLN' 'MLS' 'LN' 'LS']
  


In [11]:
data_datetime  = np.array(table.index)
data_grid  = np.array(all_stations.unique())

In [12]:
#### 2. Extract met data table for each met variable

In [13]:
all_var = pd.Series([x[1] for x in table.columns.str.split(r'_')]).unique()
print(all_var)

['Discharge' 'Precipitation' 'AirTemperature' 'SolarRadiation'
 'RelativeHumidity' 'WindDirection' 'WindSpeed' 'SnowDepth' 'SoilMoisture'
 'SoilTemperature']


In [14]:
# Extract the discharge, precipitation, airtemperature,,, tables
# Store the met grid for each met variable in data_grid_dic
# Store the met data for each met variable in data_dic
data_dic ={}
data_names = {}

for target in all_var:
    target_list = []
    for i in np.arange(len(table.columns)):
        if table.columns[i].split(r'_')[1]== target:  ### 
            target_list.append(table.columns[i])  
            
    target_table = copy.deepcopy(table[target_list])
    
    data_names[target] = target_list
    data_dic[target] = copy.deepcopy(target_table)
    
    print('Names for',target, ':',data_names[target] )

Names for Discharge : ['TL_Discharge', 'BSG_Discharge', 'LG_Discharge', 'C1E_Discharge', 'C1W_Discharge', 'C2E_Discharge', 'C2M_Discharge']
Names for Precipitation : ['BRW_Precipitation', 'LDP_Precipitation', 'SCR_Precipitation', 'TL_Precipitation', 'LW_Precipitation']
Names for AirTemperature : ['BRW_AirTemperature', 'LDP_AirTemperature', 'SCR_AirTemperature', 'TL_AirTemperature', 'LW_AirTemperature']
Names for SolarRadiation : ['BRW_SolarRadiation_regular', 'BRW_SolarRadiation_net', 'LDP_SolarRadiation_regular', 'LDP_SolarRadiation_net', 'SCR_SolarRadiation_regular', 'SCR_SolarRadiation_net', 'TL_SolarRadiation_regular', 'TL_SolarRadiation_net', 'LW_SolarRadiation_regular', 'LW_SolarRadiation_net']
Names for RelativeHumidity : ['BRW_RelativeHumidity', 'LDP_RelativeHumidity', 'SCR_RelativeHumidity', 'TL_RelativeHumidity', 'LW_RelativeHumidity']
Names for WindDirection : ['BRW_WindDirection', 'LDP_WindDirection', 'SCR_WindDirection', 'TL_WindDirection', 'LW_WindDirection']
Names for Wi

In [15]:
data_flag_dic = {}

for target in all_var:
    target_list = []
    target_grid = []
    for i in np.arange(len(table.columns)):
        if table.columns[i].split(r'_')[1]== target:  ### 
            target_list.append(table.columns[i])  
            
    target_table = copy.deepcopy(table_flag[target_list]) ###
    data_flag_dic[target] = copy.deepcopy(target_table)

In [16]:
data_flag_dic['Discharge'].head(2)

,TL_Discharge,BSG_Discharge,LG_Discharge,C1E_Discharge,C1W_Discharge,C2E_Discharge,C2M_Discharge
DateTime,,,,,,,
1999-01-01,NaN,NaN,4.0,4.0,NaN,4.0,4.0
1999-01-02,NaN,NaN,4.0,4.0,NaN,4.0,4.0


In [17]:
data_flag_dic['SoilTemperature'].head(2)

,BRW_SoilTemperature_L_5cm,BRW_SoilTemperature_L_15cm,BRW_SoilTemperature_L_30cm,BRW_SoilTemperature_L_81cm,BRW_SoilTemperature_U_5cm,BRW_SoilTemperature_U_15cm,BRW_SoilTemperature_U_30cm,BRW_SoilTemperature_U_55cm,LDP_SoilTemperature_W_2cm,LDP_SoilTemperature_W_6cm,...,LS_SoilTemperature_Pit1_30cm,LS_SoilTemperature_Pit2_2cm,LS_SoilTemperature_Pit2_15cm,LS_SoilTemperature_Pit2_30cm,LS_SoilTemperature_Pit3_2cm,LS_SoilTemperature_Pit3_15cm,LS_SoilTemperature_Pit3_30cm,LS_SoilTemperature_Pit4_2cm,LS_SoilTemperature_Pit4_15cm,LS_SoilTemperature_Pit4_30cm
DateTime,,,,,,,,,,,,,,,,,,,,,
1999-01-01,NaN,0.0,0.0,3.0,4.0,4.0,4.0,3.0,3.0,3.0,...,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0
1999-01-02,NaN,0.0,0.0,3.0,4.0,4.0,4.0,3.0,3.0,3.0,...,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0


### Dimensions

In [18]:
# dimensions must be a tuple containing dimension names (strings) that have been defined using createDimension.
# The default value is an empty tuple, which means the variable is a scalar.

In [19]:
# Create dimension for datetime and total grid points (all stations)
dim_datetime = ncid.createDimension('Dim_Datetime',len(data_datetime))
dim_grid = ncid.createDimension('Dim_Grid',len(data_grid)) 

In [20]:
data_grid

array(['TL', 'BSG', 'LG', 'C1E', 'C1W', 'C2E', 'C2M', 'BRW', 'LDP', 'SCR',
       'LW', 'HN', 'HS', 'MHN', 'MHS', 'MLN', 'MLS', 'LN', 'LS'],
      dtype=object)

In [21]:
dim_datetime

<class 'netCDF4._netCDF4.Dimension'>: name = 'Dim_Datetime', size = 7670

In [22]:
dim_grid

<class 'netCDF4._netCDF4.Dimension'>: name = 'Dim_Grid', size = 19

In [23]:
# create dimensions of grid points for each met_var 
dim_dic = {}
for met_var in all_var:
    dim_dic[met_var] = ncid.createDimension('Dim_'+met_var+'_Grid',len(data_names[met_var ]))

In [24]:
dim_dic['Discharge']

<class 'netCDF4._netCDF4.Dimension'>: name = 'Dim_Discharge_Grid', size = 7

In [25]:
dim_dic['SoilTemperature']

<class 'netCDF4._netCDF4.Dimension'>: name = 'Dim_SoilTemperature_Grid', size = 156

## Create variables

### create datetime and total grids variables

In [26]:
datetime = ncid.createVariable('Datetime',np.float64,('Dim_Datetime',), zlib = True, fill_value = False)
grid = ncid.createVariable('Grid',np.str,('Dim_Grid',), zlib = True, fill_value = False)

In [27]:
datetime

<class 'netCDF4._netCDF4.Variable'>
float64 Datetime(Dim_Datetime)
unlimited dimensions: 
current shape = (7670,)
filling off

### create Discharge and Meteorology variables

In [28]:
variable_dic = {}
for met_var in all_var:
    variable_dic[met_var] = ncid.createVariable(met_var,np.float64,('Dim_Datetime','Dim_' + met_var + '_Grid',),
                                                zlib = True, fill_value = False)

In [29]:
variable_dic['Discharge']

<class 'netCDF4._netCDF4.Variable'>
float64 Discharge(Dim_Datetime, Dim_Discharge_Grid)
unlimited dimensions: 
current shape = (7670, 7)
filling off

In [30]:
variable_dic['SoilTemperature']

<class 'netCDF4._netCDF4.Variable'>
float64 SoilTemperature(Dim_Datetime, Dim_SoilTemperature_Grid)
unlimited dimensions: 
current shape = (7670, 156)
filling off

### create Flag variables

In [31]:
flag_dic = {}
for met_var in all_var:
    flag_var_name = met_var+'_Flag'
    flag_dic[flag_var_name] = ncid.createVariable(flag_var_name ,np.float64,
                                                   ('Dim_Datetime','Dim_' + met_var + '_Grid',),
                                                        zlib = True, fill_value = False)

In [32]:
flag_dic['SoilTemperature_Flag']

<class 'netCDF4._netCDF4.Variable'>
float64 SoilTemperature_Flag(Dim_Datetime, Dim_SoilTemperature_Grid)
unlimited dimensions: 
current shape = (7670, 156)
filling off

## Write data

### write data for datetime and grid variables

In [33]:
# datetime

In [34]:
data_datetime

array(['1999-01-01T00:00:00.000000000', '1999-01-02T00:00:00.000000000',
       '1999-01-03T00:00:00.000000000', ...,
       '2019-12-29T00:00:00.000000000', '2019-12-30T00:00:00.000000000',
       '2019-12-31T00:00:00.000000000'], dtype='datetime64[ns]')

In [35]:
datetime[:] = data_datetime
grid[:] = data_grid

In [36]:
ncid.variables['Datetime'][:]

masked_array(data=[9.1514880e+17, 9.1523520e+17, 9.1532160e+17, ...,
                   1.5775776e+18, 1.5776640e+18, 1.5777504e+18],
             mask=False,
       fill_value=1e+20)

In [37]:
pd.to_datetime(ncid.variables['Datetime'][:][0])

Timestamp('1999-01-01 00:00:00')

In [38]:
# grid

In [39]:
data_grid

array(['TL', 'BSG', 'LG', 'C1E', 'C1W', 'C2E', 'C2M', 'BRW', 'LDP', 'SCR',
       'LW', 'HN', 'HS', 'MHN', 'MHS', 'MLN', 'MLS', 'LN', 'LS'],
      dtype=object)

In [40]:
ncid.variables['Grid'][:]

array(['TL', 'BSG', 'LG', 'C1E', 'C1W', 'C2E', 'C2M', 'BRW', 'LDP', 'SCR',
       'LW', 'HN', 'HS', 'MHN', 'MHS', 'MLN', 'MLS', 'LN', 'LS'],
      dtype=object)

### Write data for meteorology variables

In [41]:
for met_var in all_var:
    print(met_var)
    variable_dic[met_var][:]= data_dic[met_var].values

Discharge
Precipitation
AirTemperature
SolarRadiation
RelativeHumidity
WindDirection
WindSpeed
SnowDepth
SoilMoisture
SoilTemperature


In [42]:
data_dic['SoilTemperature'].values

array([[       nan,        nan,        nan, ..., 5.40327729, 6.12827241,
        6.49567132],
       [       nan,        nan,        nan, ..., 5.25589668, 5.99262146,
        6.36736467],
       [       nan,        nan,        nan, ..., 3.9384033 , 4.77998422,
        5.22038096],
       ...,
       [       nan,        nan,        nan, ...,        nan,        nan,
               nan],
       [       nan,        nan,        nan, ...,        nan,        nan,
               nan],
       [       nan,        nan,        nan, ...,        nan,        nan,
               nan]])

In [43]:
ncid.variables['SoilTemperature'][:]

masked_array(
  data=[[       nan,        nan,        nan, ..., 5.40327729, 6.12827241,
         6.49567132],
        [       nan,        nan,        nan, ..., 5.25589668, 5.99262146,
         6.36736467],
        [       nan,        nan,        nan, ..., 3.9384033 , 4.77998422,
         5.22038096],
        ...,
        [       nan,        nan,        nan, ...,        nan,        nan,
                nan],
        [       nan,        nan,        nan, ...,        nan,        nan,
                nan],
        [       nan,        nan,        nan, ...,        nan,        nan,
                nan]],
  mask=False,
  fill_value=1e+20)

In [44]:
ncid.variables['SoilTemperature'][:].shape

(7670, 156)

In [45]:
#on the 12134th date, the discharge data for all nine met stations 
#ncid.variables['Discharge'][12134,:]

In [46]:
#ncid.variables['Discharge'][:,3]

### Write data for flag variables

In [47]:
for met_var in all_var:
    flag_var_name = met_var+'_Flag'
    flag_dic[flag_var_name][:]= data_flag_dic[met_var].values

In [48]:
ncid.variables['Discharge_Flag'][:]

masked_array(
  data=[[nan, nan,  4., ..., nan,  4.,  4.],
        [nan, nan,  4., ..., nan,  4.,  4.],
        [nan, nan,  4., ..., nan,  4.,  4.],
        ...,
        [nan, nan,  1., ..., nan,  3.,  3.],
        [nan, nan,  1., ..., nan,  3.,  3.],
        [nan, nan,  1., ..., nan,  3.,  3.]],
  mask=False,
  fill_value=1e+20)

## Write Attributes (additional information)

### Datetime attributes (to the class)

In [49]:
# range attribute of datetime
datetime.range = [str(table.index[0]).split(' ')[0], str(table.index[-1]).split(' ')[0]]
datetime.range

['1999-01-01', '2019-12-31']

### Grid point attribues: Latitudes & Longitudes

In [50]:
ncid.variables['Grid'][:]

array(['TL', 'BSG', 'LG', 'C1E', 'C1W', 'C2E', 'C2M', 'BRW', 'LDP', 'SCR',
       'LW', 'HN', 'HS', 'MHN', 'MHS', 'MLN', 'MLS', 'LN', 'LS'],
      dtype=object)

In [51]:
ncid.variables['Grid'].latitude = ['43.73019','43.7407086','43.68834','43.7181614','43.7184233',
                                   '43.7027903','43.7022778','43.75876','43.737078','43.71105',
                                   '43.6885278','43.7406725','43.739075','43.7171517','43.7205044',
                                   '43.7015067','43.7049736','43.6892464','43.69082']

In [52]:
ncid.variables['Grid'].longitude = ['-116.140143','-116.0989983','-116.17862','-116.1372986','-116.1374192',
                                   '-116.1527722','-116.154095','-116.090404','-116.1221131','-116.09912',
                                   '-116.16991','-116.118595','-116.115775','-116.1253089','-116.1257444',
                                   '-116.1484592','-116.1497381','-116.1696892','-116.1693822']

In [54]:
ncid.variables['Grid'].elevation = ['1610','1680','1036','1335','1342','1158','1143','2114','1850','1720',
                                   '1151','1812','1835','1472','1457','1288','1298','1120','1139']

In [55]:
ncid.variables['Grid'].area = [''] * len(ncid.variables['Grid'][:])

In [56]:
# check the dimensions
a = len(ncid.variables['Grid'][:]) 
b = len(ncid.variables['Grid'].latitude)
c = len(ncid.variables['Grid'].longitude)
d = len(ncid.variables['Grid'].longitude)
e = len(ncid.variables['Grid'].area)
print(a,b,c,d,e,a==b==c==d==e)

19 19 19 19 19 True


### Meteorology variable attributes : Units & Location

In [57]:
for var in all_var:
    variable_dic[var].names = data_names[var]

In [58]:
for var in all_var:
    start = ''
    over = ''
    var_column = []
    for k in table.columns:
        if k.split('_')[1] == var:
            var_column.append(k)
    var_table = table[var_column]
    for i in range(var_table.shape[0]):
        if any(var_table.iloc[i,:].notnull()) == True: # Any one of them is true
            start = var_table.index[i]
            break
            
    for i in range(1,var_table.shape[0]):
        if any(var_table.iloc[-i,:].notnull()) == True: # Since some are empty
            over = var_table.index[-i]
            break
            
    print(var, str(start)[:-9], ',',str(over)[:-9] )
    variable_dic[var].time = [str(start)[:-9], str(over)[:-9]]

Discharge 1999-01-01 , 2019-12-31
Precipitation 1999-01-01 , 2019-12-31
AirTemperature 1999-01-01 , 2019-12-31
SolarRadiation 2000-01-01 , 2019-12-31
RelativeHumidity 1999-01-01 , 2019-12-31
WindDirection 1999-01-01 , 2019-12-31
WindSpeed 1999-01-01 , 2019-12-31
SnowDepth 1999-01-01 , 2019-12-31
SoilMoisture 1999-01-01 , 2018-05-21
SoilTemperature 1999-01-01 , 2018-05-21


In [59]:
var = 'Discharge'
variable_dic[var].units = 'L/s'
variable_dic[var].names

['TL_Discharge',
 'BSG_Discharge',
 'LG_Discharge',
 'C1E_Discharge',
 'C1W_Discharge',
 'C2E_Discharge',
 'C2M_Discharge']

In [60]:
var = 'Precipitation'
variable_dic[var].units = 'mm/d'
variable_dic[var].names

['BRW_Precipitation',
 'LDP_Precipitation',
 'SCR_Precipitation',
 'TL_Precipitation',
 'LW_Precipitation']

In [61]:
var = 'AirTemperature'
variable_dic[var].units = 'Degree celsius'
variable_dic[var].names

['BRW_AirTemperature',
 'LDP_AirTemperature',
 'SCR_AirTemperature',
 'TL_AirTemperature',
 'LW_AirTemperature']

In [62]:
var = 'SolarRadiation'
variable_dic[var].units = 'W/m2'
variable_dic[var].names

['BRW_SolarRadiation_regular',
 'BRW_SolarRadiation_net',
 'LDP_SolarRadiation_regular',
 'LDP_SolarRadiation_net',
 'SCR_SolarRadiation_regular',
 'SCR_SolarRadiation_net',
 'TL_SolarRadiation_regular',
 'TL_SolarRadiation_net',
 'LW_SolarRadiation_regular',
 'LW_SolarRadiation_net']

In [63]:
var = 'RelativeHumidity'
variable_dic[var].units = '%'
variable_dic[var].names

['BRW_RelativeHumidity',
 'LDP_RelativeHumidity',
 'SCR_RelativeHumidity',
 'TL_RelativeHumidity',
 'LW_RelativeHumidity']

In [64]:
var = 'WindDirection'
variable_dic[var].units = 'Degree'
variable_dic[var].names

['BRW_WindDirection',
 'LDP_WindDirection',
 'SCR_WindDirection',
 'TL_WindDirection',
 'LW_WindDirection']

In [65]:
var = 'WindSpeed'
variable_dic[var].units = 'm/s'
variable_dic[var].names

['BRW_WindSpeed',
 'LDP_WindSpeed',
 'SCR_WindSpeed',
 'TL_WindSpeed',
 'LW_WindSpeed']

In [66]:
var = 'SnowDepth'
variable_dic[var].units = 'mm'
variable_dic[var].names

['BRW_SnowDepth',
 'LDP_SnowDepth',
 'SCR_SnowDepth',
 'TL_SnowDepth',
 'LW_SnowDepth']

In [67]:
var = 'SoilMoisture'
variable_dic[var].units ='%'
variable_dic[var].names

['BRW_SoilMoisture_L_5cm',
 'BRW_SoilMoisture_L_15cm',
 'BRW_SoilMoisture_L_30cm',
 'BRW_SoilMoisture_L_81cm',
 'BRW_SoilMoisture_U_5cm',
 'BRW_SoilMoisture_U_15cm',
 'BRW_SoilMoisture_U_30cm',
 'BRW_SoilMoisture_U_55cm',
 'LDP_SoilMoisture_W_8cm',
 'LDP_SoilMoisture_E_8cm',
 'SCR_SoilMoisture_5cm',
 'TL_SoilMoisture_P3_5cm',
 'TL_SoilMoisture_P3_15cm',
 'TL_SoilMoisture_P3_30cm',
 'TL_SoilMoisture_P3_60cm',
 'TL_SoilMoisture_P3_100cm',
 'TL_SoilMoisture_P4_5cm',
 'TL_SoilMoisture_P4_15cm',
 'TL_SoilMoisture_P4_30cm',
 'TL_SoilMoisture_P4_45cm',
 'TL_SoilMoisture_P4_65cm',
 'LW_SoilMoisture_P1_5cm',
 'LW_SoilMoisture_P1_15cm',
 'LW_SoilMoisture_P1_30cm',
 'LW_SoilMoisture_P1_50cm',
 'LW_SoilMoisture_P1_100cm',
 'LW_SoilMoisture_P2_5cm',
 'LW_SoilMoisture_P2_15cm',
 'LW_SoilMoisture_P2_30cm',
 'LW_SoilMoisture_P2_60cm',
 'LW_SoilMoisture_P2_100cm',
 'HN_SoilMoisture_Pit1_2cm',
 'HN_SoilMoisture_Pit1_15cm',
 'HN_SoilMoisture_Pit1_30cm',
 'HN_SoilMoisture_Pit1_37cmBdrk',
 'HN_SoilMoisture

In [68]:
var = 'SoilTemperature'
variable_dic[var].units = 'Degree celsius'
variable_dic[var].names

['BRW_SoilTemperature_L_5cm',
 'BRW_SoilTemperature_L_15cm',
 'BRW_SoilTemperature_L_30cm',
 'BRW_SoilTemperature_L_81cm',
 'BRW_SoilTemperature_U_5cm',
 'BRW_SoilTemperature_U_15cm',
 'BRW_SoilTemperature_U_30cm',
 'BRW_SoilTemperature_U_55cm',
 'LDP_SoilTemperature_W_2cm',
 'LDP_SoilTemperature_W_6cm',
 'LDP_SoilTemperature_E_2cm',
 'LDP_SoilTemperature_E_6cm',
 'LDP_SoilTemperature_2.5cm',
 'LDP_SoilTemperature_20cm',
 'LDP_SoilTemperature_33cm',
 'SCR_SoilTemperature_5cm',
 'TL_SoilTemperature_P3_5cm',
 'TL_SoilTemperature_P3_30cm',
 'TL_SoilTemperature_P3_60cm',
 'TL_SoilTemperature_P3_100cm',
 'TL_SoilTemperature_P4_5cm',
 'TL_SoilTemperature_P4_15cm',
 'TL_SoilTemperature_P4_30cm',
 'TL_SoilTemperature_P4_45cm',
 'TL_SoilTemperature_P4_65cm',
 'LW_SoilTemperature_P1_5cm',
 'LW_SoilTemperature_P1_15cm',
 'LW_SoilTemperature_P1_30cm',
 'LW_SoilTemperature_P1_50cm',
 'LW_SoilTemperature_P1_100cm',
 'LW_SoilTemperature_P2_5cm',
 'LW_SoilTemperature_P2_15cm',
 'LW_SoilTemperature_P2_

### Close the file

In [69]:
ncid.variables

OrderedDict([('Datetime', <class 'netCDF4._netCDF4.Variable'>
              float64 Datetime(Dim_Datetime)
                  range: ['1999-01-01', '2019-12-31']
              unlimited dimensions: 
              current shape = (7670,)
              filling off), ('Grid', <class 'netCDF4._netCDF4.Variable'>
              vlen Grid(Dim_Grid)
                  latitude: ['43.73019', '43.7407086', '43.68834', '43.7181614', '43.7184233', '43.7027903', '43.7022778', '43.75876', '43.737078', '43.71105', '43.6885278', '43.7406725', '43.739075', '43.7171517', '43.7205044', '43.7015067', '43.7049736', '43.6892464', '43.69082']
                  longitude: ['-116.140143', '-116.0989983', '-116.17862', '-116.1372986', '-116.1374192', '-116.1527722', '-116.154095', '-116.090404', '-116.1221131', '-116.09912', '-116.16991', '-116.118595', '-116.115775', '-116.1253089', '-116.1257444', '-116.1484592', '-116.1497381', '-116.1696892', '-116.1693822']
                  elevation: ['1610', '1680', '1036

In [70]:
ncid.close()

### Read the NetCDF file

In [71]:
ncid2 = nc4.Dataset(watershed + '_NetCDF.nc', 'r')
print(ncid2)

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    dimensions(sizes): Dim_Datetime(7670), Dim_Grid(19), Dim_Discharge_Grid(7), Dim_Precipitation_Grid(5), Dim_AirTemperature_Grid(5), Dim_SolarRadiation_Grid(10), Dim_RelativeHumidity_Grid(5), Dim_WindDirection_Grid(5), Dim_WindSpeed_Grid(5), Dim_SnowDepth_Grid(5), Dim_SoilMoisture_Grid(152), Dim_SoilTemperature_Grid(156)
    variables(dimensions): float64 Datetime(Dim_Datetime), <class 'str'> Grid(Dim_Grid), float64 Discharge(Dim_Datetime,Dim_Discharge_Grid), float64 Precipitation(Dim_Datetime,Dim_Precipitation_Grid), float64 AirTemperature(Dim_Datetime,Dim_AirTemperature_Grid), float64 SolarRadiation(Dim_Datetime,Dim_SolarRadiation_Grid), float64 RelativeHumidity(Dim_Datetime,Dim_RelativeHumidity_Grid), float64 WindDirection(Dim_Datetime,Dim_WindDirection_Grid), float64 WindSpeed(Dim_Datetime,Dim_WindSpeed_Grid), float64 SnowDepth(Dim_Datetime,Dim_SnowDepth_Grid), float64 SoilMoisture(Dim_Datetime,

In [72]:
print(ncid2.variables['Discharge'])

<class 'netCDF4._netCDF4.Variable'>
float64 Discharge(Dim_Datetime, Dim_Discharge_Grid)
    names: ['TL_Discharge', 'BSG_Discharge', 'LG_Discharge', 'C1E_Discharge', 'C1W_Discharge', 'C2E_Discharge', 'C2M_Discharge']
    time: ['1999-01-01', '2019-12-31']
    units: L/s
unlimited dimensions: 
current shape = (7670, 7)
filling off



In [73]:
ncid2.variables

OrderedDict([('Datetime', <class 'netCDF4._netCDF4.Variable'>
              float64 Datetime(Dim_Datetime)
                  range: ['1999-01-01', '2019-12-31']
              unlimited dimensions: 
              current shape = (7670,)
              filling off), ('Grid', <class 'netCDF4._netCDF4.Variable'>
              vlen Grid(Dim_Grid)
                  latitude: ['43.73019', '43.7407086', '43.68834', '43.7181614', '43.7184233', '43.7027903', '43.7022778', '43.75876', '43.737078', '43.71105', '43.6885278', '43.7406725', '43.739075', '43.7171517', '43.7205044', '43.7015067', '43.7049736', '43.6892464', '43.69082']
                  longitude: ['-116.140143', '-116.0989983', '-116.17862', '-116.1372986', '-116.1374192', '-116.1527722', '-116.154095', '-116.090404', '-116.1221131', '-116.09912', '-116.16991', '-116.118595', '-116.115775', '-116.1253089', '-116.1257444', '-116.1484592', '-116.1497381', '-116.1696892', '-116.1693822']
                  elevation: ['1610', '1680', '1036

#### Look at all the important attributes

In [74]:
# value
ncid2.variables['SoilTemperature'][:]

masked_array(
  data=[[       nan,        nan,        nan, ..., 5.40327729, 6.12827241,
         6.49567132],
        [       nan,        nan,        nan, ..., 5.25589668, 5.99262146,
         6.36736467],
        [       nan,        nan,        nan, ..., 3.9384033 , 4.77998422,
         5.22038096],
        ...,
        [       nan,        nan,        nan, ...,        nan,        nan,
                nan],
        [       nan,        nan,        nan, ...,        nan,        nan,
                nan],
        [       nan,        nan,        nan, ...,        nan,        nan,
                nan]],
  mask=False,
  fill_value=1e+20)

In [75]:
# unit
ncid2.variables['SoilTemperature'].units

'Degree celsius'

In [76]:
# names
ncid2.variables['SoilTemperature'].names

['BRW_SoilTemperature_L_5cm',
 'BRW_SoilTemperature_L_15cm',
 'BRW_SoilTemperature_L_30cm',
 'BRW_SoilTemperature_L_81cm',
 'BRW_SoilTemperature_U_5cm',
 'BRW_SoilTemperature_U_15cm',
 'BRW_SoilTemperature_U_30cm',
 'BRW_SoilTemperature_U_55cm',
 'LDP_SoilTemperature_W_2cm',
 'LDP_SoilTemperature_W_6cm',
 'LDP_SoilTemperature_E_2cm',
 'LDP_SoilTemperature_E_6cm',
 'LDP_SoilTemperature_2.5cm',
 'LDP_SoilTemperature_20cm',
 'LDP_SoilTemperature_33cm',
 'SCR_SoilTemperature_5cm',
 'TL_SoilTemperature_P3_5cm',
 'TL_SoilTemperature_P3_30cm',
 'TL_SoilTemperature_P3_60cm',
 'TL_SoilTemperature_P3_100cm',
 'TL_SoilTemperature_P4_5cm',
 'TL_SoilTemperature_P4_15cm',
 'TL_SoilTemperature_P4_30cm',
 'TL_SoilTemperature_P4_45cm',
 'TL_SoilTemperature_P4_65cm',
 'LW_SoilTemperature_P1_5cm',
 'LW_SoilTemperature_P1_15cm',
 'LW_SoilTemperature_P1_30cm',
 'LW_SoilTemperature_P1_50cm',
 'LW_SoilTemperature_P1_100cm',
 'LW_SoilTemperature_P2_5cm',
 'LW_SoilTemperature_P2_15cm',
 'LW_SoilTemperature_P2_

In [77]:
# Grid
ncid2.variables['Grid'][:]

array(['TL', 'BSG', 'LG', 'C1E', 'C1W', 'C2E', 'C2M', 'BRW', 'LDP', 'SCR',
       'LW', 'HN', 'HS', 'MHN', 'MHS', 'MLN', 'MLS', 'LN', 'LS'],
      dtype=object)

In [78]:
# Grid
ncid2.variables['Grid']

<class 'netCDF4._netCDF4.Variable'>
vlen Grid(Dim_Grid)
    latitude: ['43.73019', '43.7407086', '43.68834', '43.7181614', '43.7184233', '43.7027903', '43.7022778', '43.75876', '43.737078', '43.71105', '43.6885278', '43.7406725', '43.739075', '43.7171517', '43.7205044', '43.7015067', '43.7049736', '43.6892464', '43.69082']
    longitude: ['-116.140143', '-116.0989983', '-116.17862', '-116.1372986', '-116.1374192', '-116.1527722', '-116.154095', '-116.090404', '-116.1221131', '-116.09912', '-116.16991', '-116.118595', '-116.115775', '-116.1253089', '-116.1257444', '-116.1484592', '-116.1497381', '-116.1696892', '-116.1693822']
    elevation: ['1610', '1680', '1036', '1335', '1342', '1158', '1143', '2114', '1850', '1720', '1151', '1812', '1835', '1472', '1457', '1288', '1298', '1120', '1139']
    area: ['', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '']
vlen data type: <class 'str'>
unlimited dimensions: 
current shape = (19,)

In [79]:
len(ncid2.variables)

22

In [80]:
ncid2.close()